In [1]:
import sys
import logging

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms, datasets
import numpy as np
import utils
from model import VGG16_BN

logging.basicConfig(stream=sys.stdout, level=logging.INFO,
                    format="%(asctime)s %(message)s", datefmt="%m-%d %H:%M")

In [2]:
def get_dataloader():
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    test_transforms = transforms.Compose([
        transforms.ToTensor(),
        normalize])

    test_dataset = datasets.CIFAR10(root="/root/dataset/CIFAR", train=False, transform=test_transforms, download=True)
    test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False, num_workers=0, pin_memory=True)
    return test_loader

In [3]:
from utils import *
import time
from torch.autograd import Variable

# Validation function
def validate(val_loader, model, cuda=False):
    batch_time = AverageMeter('Time', ':6.3f')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(len(val_loader), batch_time, top1, top5, prefix='Test: ')

    model.eval()
    with torch.no_grad():
        end = time.time()
        for i, (input, target) in enumerate(val_loader):
            end = time.time()
            if cuda:
                input = Variable(input).cuda()
                target = Variable(target).cuda()
            else:
                input = Variable(input)
                target = Variable(target)

            output = model(input)

            acc1, acc5 = accuracy(output, target, topk=(1,5))
            top1.update(acc1[0], input.size(0))
            top5.update(acc5[0], input.size(0))

            batch_time.update(time.time() - end)

            if i % 10 == 0:
                progress.print(i)
        logging.info('====> Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'.format(top1=top1, top5=top5))

    return top1.avg, top5.avg

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
cuda = torch.cuda.is_available()

# Load Models
model = torchvision.models.vgg16_bn(pretrained=True)
model.to(device)

# Get Data
loader = get_dataloader()
top1, top5 = validate(loader, model)

Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth



Files already downloaded and verified
Test: [ 0/40]	Time  4.177 ( 4.177)	Acc@1   0.00 (  0.00)	Acc@5   0.00 (  0.00)
Test: [10/40]	Time  4.885 ( 4.212)	Acc@1   0.00 (  0.00)	Acc@5   0.00 (  0.28)
Test: [20/40]	Time  3.468 ( 4.236)	Acc@1   0.39 (  0.02)	Acc@5   0.78 (  0.22)
Test: [30/40]	Time  3.573 ( 4.288)	Acc@1   0.00 (  0.01)	Acc@5   0.00 (  0.19)
11-02 04:58 ====> Acc@1 0.030 Acc@5 0.190


In [5]:
# 가설 설정
"""가설
1. Target Neuron은 목표한 공격 클래스에 대해서 activation이 크다.
2. Target Neuron은 그 외의 클래스에 대해서도 높은 경향성을 보인다.
3. 따라서 모든 클래스에서 공통적으로 높은 activation을 보이는 neuron은
target neuron일 가능성이 높다.
4. 모든 클래스에 대한 activation을 사용해 target neuron을 특정한다.
"""

"""
P_t = SUM(a_c x (PI(Activation_n)))
- a_c : prior value for the class to be target class ==> calculated by average activation of class
- Activation_n : activation of neuron n
"""

"""
사전 작업
1. AVG(Class_Activation) 과 target 과의 상관관계
2. PI(Activation_n) 과 target_neuron과의 상관관계
"""


'\n사전 작업\n1. AVG(Class_Activation) 과 target 과의 상관관계\n2. PI(Activation_n) 과 target_neuron과의 상관관계\n'